# Transcription Factor Project - Differential Expression Analysis and Minimum Distortion Embedding (Pipeline Steps G-I)
**Robin Anwyl, UCSD Subramaniam Lab**

**Project Goal:** Analyze the hiPSC Perturb-seq dataset from the Mali lab (Nourreddine et al preprint) to investigate the effects of transcription factor knockouts (TF KOs)

**Notebook Description:** 
-  Dataset: QC'd TF KO (and NTC) dataset
-  Analysis: pseudobulk differential expression analysis (DEA), pairwise Pearson correlation matrix, minimum distortion embedding (MDE)
***

# Import statements

In [ ]:
# Using psp_env virtual environment
import sys
import os
repo_root = "/home/ranwyl/KOLF2.1J_Perturbation_Cell_Atlas/"
if repo_root not in sys.path:
    sys.path.insert(0, "/home/ranwyl/KOLF2.1J_Perturbation_Cell_Atlas/")

import psp
import gc
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'

# Differential expression analysis - partitioning method

We will carry out differential expression analysis at the gRNA level with PyDESeq2 using a pseudobulk method. The cells for each gRNA are partitioned evenly into pseudoreplicates and compared to an equal number of NTC cells.

In [ ]:
import anndata as ad
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm_joblib import tqdm_joblib
from joblib import Parallel, delayed
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.default_inference import DefaultInference
import gc

def generate_pseudoreplicates_for_DE(adata: ad.AnnData, 
                                     target_value: str, 
                                     ntc_cell_indices: pd.Index, 
                                     rng: np.random.Generator, 
                                     target_column: str = "perturbation", 
                                     layer: str = "counts"):
    """
    Generate independent pseudoreplicates for a given target (gRNA or gene target) 
    and matched NTC cells. Manually set NTC as reference for DE analysis.
    """
    # Create views for target and NTC cells
    if target_column not in adata.obs:
        print(f"Error: {target_column} not in adata.obs")
        return
    target_mask = adata.obs[target_column] == target_value
    target_view = adata[target_mask]
    ntc_view = adata[ntc_cell_indices]

    # Get data matrices from counts layer
    if layer in adata.layers:
        target_data = target_view.layers[layer]
        ntc_data = ntc_view.layers[layer]
    else:
        target_data = target_view.X
        ntc_data = ntc_view.X        
    
    # Convert to dense if sparse
    if sparse.issparse(target_data):
        target_data = target_data.toarray()
    if sparse.issparse(ntc_data):
        ntc_data = ntc_data.toarray()

    # Calculate number of cells to sample
    n_target_cells = target_data.shape[0]
    n_ntc_cells = ntc_data.shape[0]
    n_reps = 2 if n_target_cells <= 35 else 3
    target_rep_size = n_target_cells // n_reps
    ntc_rep_size = min(target_rep_size, n_ntc_cells // n_reps)
    if ntc_rep_size < target_rep_size:
        print("Warning: Unmatched number of target and NTC cells")

    # Pre-allocate arrays for results
    target_bulk = np.zeros((n_reps, target_data.shape[1]), dtype=np.int64)
    ntc_bulk = np.zeros((n_reps, ntc_data.shape[1]), dtype=np.int64)

    # Sample cells for all replicates
    target_sample_size = target_rep_size * n_reps
    ntc_sample_size = ntc_rep_size * n_reps
    sampled_target_indices = \
        rng.choice(n_target_cells, target_sample_size, replace=False)
    sampled_ntc_indices = \
        rng.choice(n_ntc_cells, ntc_sample_size, replace=False)
    # Generate replicates using vectorized operations
    for i in range(n_reps):
        # Sample indices
        target_start, target_stop = i*target_rep_size, (i+1)*target_rep_size
        target_rep_indices = sampled_target_indices[target_start:target_stop]
        ntc_start, ntc_stop = i*ntc_rep_size, (i+1)*ntc_rep_size
        ntc_rep_indices = sampled_ntc_indices[ntc_start:ntc_stop]

        # Calculate sums using vectorized operations
        target_bulk[i] = np.sum(target_data[target_rep_indices], axis=0).astype(np.int64)
        ntc_bulk[i] = np.sum(ntc_data[ntc_rep_indices], axis=0).astype(np.int64)  

    # Create sample names
    if "_" in target_value:
        target_value = target_value.replace("_", "-")
    sample_names = [f"{target_value}-rep{i+1}" for i in range(n_reps)]
    control_names = [f"NTC-rep{i+1}" for i in range(n_reps)]
    
    # Combine data
    combined_data = np.vstack([target_bulk, ntc_bulk])
    combined_names = sample_names + control_names
    
    # Create metadata DF
    metadata_df = pd.DataFrame({
        'condition': [target_value] * n_reps + ['NTC'] * n_reps
    }, index=combined_names)
    # Set NTC as reference for DESeq2
    metadata_df["condition"] = pd.Categorical(
        metadata_df["condition"],
        categories=["NTC", target_value],
        ordered=True
        )
    
    # Create counts DF
    counts_df = pd.DataFrame(
        combined_data,
        index=combined_names,
        columns=adata.var_names
    )

    return counts_df, metadata_df

def differential_expression(adata: ad.AnnData, 
                    target_column: str = "perturbation", 
                    ntc_cells_delimiter: str = "NTC", 
                    alpha: float = 0.05, n_cpus: int = 20, 
                    layer: str = "counts", de_rng=None, 
                    debug: bool = False):
    """
    Run differential expression analysis on each perturbation in the dataset.
    de_rng takes a numpy random Generator object.
    """
    # Identify KD and NTC cells
    perturbations = list(adata.obs[target_column].unique())
    perturbations.remove(ntc_cells_delimiter) # Remove NTC group
    if debug == True: # Debug mode: run with 5 perturbations
        print(f"Running in debug mode with 5 perturbations")
        perturbations = perturbations[:5]
    ntc_cell_indices = np.where(adata.obs[perturbations] == ntc_cells_delimiter)[0]

    # Create RNG object to handle case where one is not provided
    if de_rng is None:
        hard_coded_seed = 42
        de_rng = np.random.default_rng(hard_coded_seed)
    # Create one child RNG object per perturbation
    streams = de_rng.spawn(len(perturbations))

    # Determine how many CPUs to use per joblib Parallel job
    #   and per DE analysis run
    if n_cpus < 3:
        n_cpus_for_DE = n_cpus
    elif n_cpus <= 10:
        n_cpus_for_DE = 3
    elif 20 <= n_cpus < 30:
        n_cpus_for_DE = 4
    elif 30 <= n_cpus < 50:
        n_cpus_for_DE = 5
    else:
        n_cpus_for_DE = 6
    n_jobs = max(1, n_cpus // n_cpus_for_DE)

    # Warning if given layer not found
    if layer not in adata.layers:
         print(f"Warning: {layer} not in adata.layers, using adata.X instead")

    # Function to run DE analysis on a single perturbation
    def process_perturbation(target_value: str, rng: np.random.Generator):
        # Generate pseudoreplicates
        pseudo_bulk_df, metadata_df = generate_pseudoreplicates_for_DE(
            adata, target_value, ntc_cell_indices, rng, 
            target_column=target_column, layer=layer
        )

        # Set number of CPUs to use for each DE run
        inference = DefaultInference(n_cpus=n_cpus_for_DE) 

        # Read counts modeling and fitting dispersions
        dds = DeseqDataSet(
            counts = pseudo_bulk_df, 
            metadata = metadata_df,
            refit_cooks=True,
            inference=inference,
            quiet=True
            )
        dds.deseq2()
        
        # Statistical testing
        target_hyphenated = target_value.replace('_', '-')
        contrast = ["condition", target_hyphenated, ntc_cells_delimiter]
        stat_res = DeseqStats(
            dds, 
            contrast=contrast, 
            alpha=alpha,
            inference=inference,
            quiet=True)
        stat_res.summary()
        results_no_shrink = stat_res.results_df
        # Shrink LFCs for downstream analysis
        stat_res.lfc_shrink(coeff=f"condition[T.{contrast[1]}]", adapt=False)
        results_shrink = stat_res.results_df

        # Clean up memory
        del pseudo_bulk_df, metadata_df, dds, stat_res
        gc.collect()

        # Return results with and without LFC shrinkage
        return results_no_shrink, results_shrink

    # Run DE analysis on all perturbations in parallel
    with tqdm_joblib(desc="Running DE analysis", total=len(perturbations)):
        results_no_shrink, results_shrink = Parallel(n_jobs=n_jobs)(
            delayed(process_perturbation)(target_value, rng)
            for target_value, rng in zip(perturbations, streams))
    
    results_no_shrink_dict = dict(zip(perturbations, results_no_shrink))
    results_shrink_dict = dict(zip(perturbations, results_shrink))
    
    return results_no_shrink_dict, results_shrink_dict

def build_de_df(results_dict: dict):
    """
    Build DF of all DE results: baseMean, log2FoldChange, lfcSE, 
    stat (LFC divided by LFC SE), pvalue, padj.
    """
    df_list = list()
    for gene_target, df in results_dict.items():
        # Sort genes alphabetically
        df = df.sort_index()
        # Prepend perturbation name to each column label
        new_col_names = {col: f"{gene_target}_{col}" for col in df}
        df = df.rename(columns=new_col_names)
        df_list.append(df)
    if len(df_list) > 1:
        # Concatenate all DE result DataFrames (using intersection of genes)
        de_df = pd.concat(df_list, axis=1)
        de_df = de_df.sort_index()
    return de_df

def build_deg_df(de_df: pd.DataFrame, 
                 perturbation: str, 
                 lfc_threshold: float = 0, 
                 padj_threshold: float = 0.05):
    """
    Return DE results filtered to only the given perturbation and only
    genes that pass LFC and p-adj thresholds.
    """
    perturbation_cols = de_df.columns[de_df.columns.str.contains(perturbation)]
    perturbation_df = de_df[perturbation_cols]
    perturbation_deg_df = \
        perturbation_df[
            (abs(perturbation_df[f"{perturbation}_log2FoldChange"]) > lfc_threshold) 
            & (perturbation_df[f"{perturbation}_padj"] < padj_threshold)
            ]
    return perturbation_deg_df

def benchmark_NTC_FDR(
    adata: ad.AnnData,
    gRNA_column: str = "gRNA",
    ntc_cells_delimiter: str = "Non-Targeting",
    layer: str = "counts",
    alpha: float = 0.05,
    n_cpus: int = 16,
    debug: bool = True
):
    """
    Benchmark FDR control by comparing NTC sgRNAs against each other.
    For each NTC sgRNA, run DE analysis against all other NTC sgRNAs.
    Calculate distribution of DEGs (FDR < 0.05) per NTC sgRNA. 
    Determine threshold at which 95% of NTC sgRNAs have fewer DEGs.
    Add a column to adata.obs indicating if each perturbation exceeds
    the NTC FDR threshold. Not batch-aware. No results stored in adata.

    gRNA_column: adata.obs column with identifier for each sgRNA, default "gRNA"
    ntc_cells_delimiter: prefix of NTC sgRNA in gRNA column, default "Non-Targeting"
    layer: AnnData object layer to use for DE analysis, default "counts"
    alpha: significance threshold for DESeq2, default 0.05
    """
    # Get AnnData with only NTC cells based on gRNA_column and delimiter
    ntc_mask = adata.obs[gRNA_column].astype(str).str.contains(ntc_cells_delimiter)
    if not ntc_mask.any():
        raise ValueError(f"No NTC cells found in {gRNA_column} containing "
                         f"'{ntc_cells_delimiter}'")
    ntc_adata = adata[ntc_mask].copy()

    # Get unique NTC sgRNAs
    if gRNA_column not in ntc_adata.obs.columns:
        raise ValueError(f"Column '{gRNA_column}' not found in adata.obs")
    ntc_sgRNAs = list(ntc_adata.obs[gRNA_column].unique())
    if len(ntc_sgRNAs) < 3:
        print(f"Not enough unique NTC sgRNAs for benchmarking (need at least 3, found {len(ntc_sgRNAs)})")
        return
    if debug == True:
        print("Running in debug mode with 5 NTC sgRNA")
        ntc_sgRNAs = ntc_sgRNAs[:5]

    # Create RNG object to handle case where one is not provided
    if de_rng is None:
        hard_coded_seed = 42
        de_rng = np.random.default_rng(hard_coded_seed)
    # Create one child RNG object per NTC sgRNA
    streams = de_rng.spawn(len(ntc_sgRNAs))

    # Determine how many CPUs to use per joblib Parallel job
    #   and per DE analysis run
    if n_cpus < 3:
        n_cpus_for_DE = n_cpus
    elif n_cpus <= 10:
        n_cpus_for_DE = 3
    elif 20 <= n_cpus < 30:
        n_cpus_for_DE = 4
    elif 30 <= n_cpus < 50:
        n_cpus_for_DE = 5
    else:
        n_cpus_for_DE = 6
    n_jobs = max(1, n_cpus // n_cpus_for_DE)
    
    # Warning if given layer not found
    if layer not in adata.layers:
        print(f"Warning: {layer} not found in adata.layers, using adata.X instead")

    # Helper function
    def process_ntc_gRNA(target_ntc_gRNA: str,
                         rng: np.random.Generator):
        # Check if there are enough target cells
        target_cells = ntc_adata[ntc_adata.obs[gRNA_column] == target_ntc_gRNA].obs.index
        if len(target_cells) < 10:
            print(f"Skipping {target_ntc_gRNA}: too few cells ({len(target_cells)})")
            return
        
        # Get other NTC cells (excluding the target gRNA)
        other_ntc_cells = ntc_adata[ntc_adata.obs[gRNA_column] != target_ntc_gRNA].obs.index
        if len(other_ntc_cells) < 10:
            print(f"Skipping {target_ntc_gRNA}: too few other NTC cells ({len(other_ntc_cells)})")
            return
        
        # Temporarily create "perturbation" column to use with generate_pseudoreplicates_for_DE
        # This labels target gRNA cells as the "perturbation" and other NTC cells as "NTC"
        ntc_adata_temp = ntc_adata.copy()
        ntc_adata_temp.obs["temp_pert"] = "NTC"
        ntc_adata_temp.obs.loc[ntc_adata_temp.obs[gRNA_column] == target_ntc_gRNA, 'temp_pert'] = target_ntc_gRNA
        
        # Generate pseudoreplicates
        pseudo_bulk_df, metadata_df = generate_pseudoreplicates_for_DE(
            ntc_adata_temp, target_ntc_gRNA, other_ntc_cells, rng, 
            target_column="temp_pert", layer=layer
        )

        # Set number of CPUs to use for each DE run
        inference = DefaultInference(n_cpus=n_cpus_for_DE) 

        # Read counts modeling and fitting dispersions
        dds = DeseqDataSet(
            counts = pseudo_bulk_df, 
            metadata = metadata_df,
            refit_cooks=True,
            inference=inference,
            quiet=True
            )
        dds.deseq2()
        
        # Statistical testing
        target_hyphenated = target_ntc_gRNA.replace('_', '-')
        contrast = ["condition", target_hyphenated, ntc_cells_delimiter]
        stat_res = DeseqStats(
            dds, 
            contrast=contrast, 
            alpha=alpha,
            inference=inference,
            quiet=True)
        stat_res.summary()
        de_results = stat_res.results_df

        # Clean up memory
        del pseudo_bulk_df, metadata_df, dds, res, ntc_adata_temp
        gc.collect()

        return de_results

    # Run DE analysis on all NTC sgRNA in parallel
    with tqdm_joblib(desc="Running DE analysis", total=len(ntc_sgRNAs)):
        de_results = Parallel(n_jobs=n_jobs)(
            delayed(process_ntc_gRNA)(target_value, rng) 
            for target_value, rng in zip(ntc_sgRNAs, streams))
    results_dict = dict(zip(ntc_sgRNAs, de_results))

    # Check for successful comparisons
    if not results_dict:
        print("No successful comparisons. Check if the data layer contains integer counts.")
        return
    
    # Calculate DEGs per sgRNA-batch (handles sgRNA names with underscores)
    deg_counts = {}
    for key, result_df in results_dict.items():
        if result_df is None:
            continue
        count_key = key
        # Count DEGs for this comparison
        deg_count = sum((result_df['padj'] < alpha) & pd.notna(result_df['padj']))
        deg_counts[count_key] = deg_count

    # Check if we have any DEG counts
    if not deg_counts:
        print("No DEGs found in any comparison.")
        return

    # Create results DataFrame
    benchmark_results = pd.DataFrame.from_dict(deg_counts, orient='index', columns=['n_DEGs'])
    benchmark_results['sgRNA'] = benchmark_results.index

    benchmark_results.index.name = 'key'
    benchmark_results = benchmark_results.sort_values('n_DEGs', ascending=False)
    
    # Calculate 95th percentile threshold (FDR 0.05)
    if len(benchmark_results) > 0:
        deg_threshold = np.percentile(benchmark_results['n_DEGs'].values, 95)
    else:
        print("No results to calculate percentile.")
        return
    
    # Print statistics
    print(f"NTC Benchmark Statistics:")
    print(f"  • FDR 0.05 threshold: {int(deg_threshold)} DEGs")
    print(f"  • NTC comparisons analyzed: {len(benchmark_results)}")
    print(f"  • Mean DEGs per comparison: {benchmark_results['n_DEGs'].mean():.1f}")
    print(f"  • Median DEGs per comparison: {benchmark_results['n_DEGs'].median():.1f}")
    print(f"  • 95% of comparisons have < {int(deg_threshold)} DEGs")

    return int(deg_threshold), benchmark_results


### Test

In [ ]:
test_gene_targets = ["POU5F1", "NANOG", "NTC"]
adata_test = adata_alpha[adata_alpha.obs.gene_target.isin(test_gene_targets)].copy()
adata_test

AnnData object with n_obs × n_vars = 14034 × 20200
    obs: 'gRNA', 'n_gRNA', 'n_gRNA_UMIs', 'gene_target', 'celltype', 'perturbation_type', 'n_UMI_counts', 'n_genes', 'perturbed', 'channel', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'run', 'gene_target_ensembl_id', 'gene_target_expression (CPM)', 'NTC_target_gene_expression (CPM)', 'target_knockdown', 'target_knockdown_z_score', 'ed_category', 'anomaly_score'
    var: 'gene_ids', 'feature_types', 'n_UMI_counts', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: '

In [ ]:
adata_test.obs.gene_target.unique().tolist()

['NTC', 'POU5F1', 'NANOG']

In [ ]:
adata_test[adata_test.obs.gene_target == "POU5F1"].shape[0]

96

In [ ]:
adata_test[adata_test.obs.gene_target == "NANOG"].shape[0]

96

In [ ]:
results_dict_test_2reps = de_analysis_dataset(adata_test, n_cpus=25, n_reps=2)

Running DE analysis:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()


In [ ]:
test_de_df_2reps = build_de_df(results_dict_test_2reps)
test_de_df_2reps.head()

,POU5F1_baseMean,POU5F1_log2FoldChange,POU5F1_lfcSE,POU5F1_stat,POU5F1_pvalue,POU5F1_padj,NANOG_baseMean,NANOG_log2FoldChange,NANOG_lfcSE,NANOG_stat,NANOG_pvalue,NANOG_padj
A1BG,3.971359,-0.315204,1.012120,-0.635116,0.525353,NaN,2.959269,-0.182179,1.008075,-0.376555,0.706505,NaN
A1BG-AS1,0.229447,0.154118,1.967397,0.309621,0.756849,NaN,0.986616,-0.010507,1.227681,-0.028067,0.977609,NaN
A2M,0.498122,-0.019972,1.281926,-0.057165,0.954413,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2M-AS1,0.229447,0.154118,1.967397,0.309621,0.756849,NaN,0.239337,0.100094,2.483595,0.261954,0.793357,NaN
A2ML1,1.524100,-0.519355,1.206868,-1.133351,0.257067,NaN,1.985454,-0.216242,1.079070,-0.473599,0.635786,NaN


In [ ]:
pou5f1_df_2reps = build_deg_df(test_de_df_2reps, "POU5F1", lfc_threshold=0)
print(pou5f1_df_2reps.shape[0])
nanog_df_2reps = build_deg_df(test_de_df_2reps, "NANOG", lfc_threshold=0)
print(nanog_df_2reps.shape[0])

18
41


In [ ]:
results_dict_test_3reps = de_analysis_dataset(adata_test, n_cpus=25, n_reps=3)

Running DE analysis:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
test_de_df_3reps = build_de_df(results_dict_test_3reps)
test_de_df_3reps.head()

,POU5F1_baseMean,POU5F1_log2FoldChange,POU5F1_lfcSE,POU5F1_stat,POU5F1_pvalue,POU5F1_padj,NANOG_baseMean,NANOG_log2FoldChange,NANOG_lfcSE,NANOG_stat,NANOG_pvalue,NANOG_padj
A1BG,2.638843,-0.426730,0.892003,-0.801785,0.422677,NaN,1.954981,-0.215939,0.973807,-0.432866,0.665112,NaN
A1BG-AS1,0.149793,0.122497,2.188104,0.185412,0.852906,NaN,0.645030,-0.015579,1.188210,-0.037305,0.970242,NaN
A2M,0.321964,-0.017672,1.315148,-0.044368,0.964611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2M-AS1,0.154794,0.123601,2.192710,0.185412,0.852906,NaN,0.153256,0.126667,2.179549,0.205352,0.837297,NaN
A2ML1,1.014696,-0.541940,1.209727,-1.124439,0.260827,NaN,1.316343,-0.242587,1.043105,-0.506768,0.612318,NaN


In [ ]:
pou5f1_df_3reps = build_deg_df(test_de_df_3reps, "POU5F1", lfc_threshold=0)
print(pou5f1_df_3reps.shape[0])
nanog_df_3reps = build_deg_df(test_de_df_3reps, "NANOG", lfc_threshold=0)
print(nanog_df_3reps.shape[0])

443
64


Remove genes expressed in <100 cells

In [ ]:
adata = ad.concat([adata_alpha, adata_beta])
adata

AnnData object with n_obs × n_vars = 77912 × 20200
    obs: 'gRNA', 'n_gRNA', 'n_gRNA_UMIs', 'gene_target', 'celltype', 'perturbation_type', 'n_UMI_counts', 'n_genes', 'perturbed', 'channel', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'run', 'gene_target_ensembl_id', 'gene_target_expression (CPM)', 'NTC_target_gene_expression (CPM)', 'target_knockdown', 'target_knockdown_z_score', 'ed_category', 'anomaly_score'
    obsm: 'X_pca'
    layers: 'counts', 'normalized_counts'

In [ ]:
adata_genes_filt = adata.copy()
sc.pp.filter_genes(adata_genes_filt, min_cells=100)

filtered out 311 genes that are detected in less than 100 cells


In [ ]:
adata_genes_filt

AnnData object with n_obs × n_vars = 77912 × 19889
    obs: 'gRNA', 'n_gRNA', 'n_gRNA_UMIs', 'gene_target', 'celltype', 'perturbation_type', 'n_UMI_counts', 'n_genes', 'perturbed', 'channel', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'run', 'gene_target_ensembl_id', 'gene_target_expression (CPM)', 'NTC_target_gene_expression (CPM)', 'target_knockdown', 'target_knockdown_z_score', 'ed_category', 'anomaly_score'
    var: 'n_cells'
    obsm: 'X_pca'
    layers: 'counts', 'normalized_counts'

In [ ]:
adata_alpha_filt = adata_genes_filt[adata_genes_filt.obs.run == "ALPHA"].copy()
adata_beta_filt = adata_genes_filt[adata_genes_filt.obs.run == "BETA"].copy()

In [ ]:
test_gene_targets = ["POU5F1", "NANOG", "NTC"]
adata_test_filt = adata_alpha_filt[adata_alpha_filt.obs.gene_target.isin(test_gene_targets)].copy()
adata_test_filt

AnnData object with n_obs × n_vars = 14034 × 19889
    obs: 'gRNA', 'n_gRNA', 'n_gRNA_UMIs', 'gene_target', 'celltype', 'perturbation_type', 'n_UMI_counts', 'n_genes', 'perturbed', 'channel', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'run', 'gene_target_ensembl_id', 'gene_target_expression (CPM)', 'NTC_target_gene_expression (CPM)', 'target_knockdown', 'target_knockdown_z_score', 'ed_category', 'anomaly_score'
    var: 'n_cells'
    obsm: 'X_pca'
    layers: 'counts', 'normalized_counts'

In [ ]:
results_dict_test_2reps_filt = de_analysis_dataset(adata_test_filt, n_cpus=25, n_reps=2)

Running DE analysis:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()


In [ ]:
test_de_df_2reps_filt = build_de_df(results_dict_test_2reps_filt)
pou5f1_df_2reps = build_deg_df(test_de_df_2reps_filt, "POU5F1", lfc_threshold=0)
print(pou5f1_df_2reps.shape[0])
nanog_df_2reps = build_deg_df(test_de_df_2reps_filt, "NANOG", lfc_threshold=0)
print(nanog_df_2reps.shape[0])

19
41


In [ ]:
results_dict_test_3reps_filt = de_analysis_dataset(adata_test_filt, n_cpus=25, n_reps=3)

Running DE analysis:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
test_de_df_3reps_filt = build_de_df(results_dict_test_3reps_filt)
pou5f1_df_3reps = build_deg_df(test_de_df_3reps_filt, "POU5F1", lfc_threshold=0)
print(pou5f1_df_3reps.shape[0])
nanog_df_3reps = build_deg_df(test_de_df_3reps_filt, "NANOG", lfc_threshold=0)
print(nanog_df_3reps.shape[0])

446
64


Batch BETA

In [ ]:
def build_deg_df(de_df, tf_kd, lfc_threshold=0, padj_threshold=0.05):
    tf_kd_cols = de_df.columns[de_df.columns.str.contains(tf_kd)]
    tf_kd_df = de_df[tf_kd_cols]
    tf_kd_deg_df = tf_kd_df[(abs(tf_kd_df[f"{tf_kd}_log2FoldChange"]) > lfc_threshold) 
                            & (tf_kd_df[f"{tf_kd}_padj"] < padj_threshold)]
    return tf_kd_deg_df

def count_degs(de_df, tf_kd, lfc_threshold=0, padj_threshold=0.05):
    deg_df = build_deg_df(de_df, tf_kd, lfc_threshold, padj_threshold)
    return deg_df.shape[0]

def count_degs_for_tf_kd_list(de_df, tf_kd_list, lfc_threshold=0, padj_threshold=0.05):
    for tf_kd in tf_kd_list:
        n_degs = count_degs(de_df, tf_kd, lfc_threshold, padj_threshold)
        if n_degs == 1:
            print(f"{tf_kd} has {n_degs} DEG")
        else:
            print(f"{tf_kd} has {n_degs} DEGs")

In [ ]:
beta_kds = adata_beta_filt.obs.gene_target.unique().to_list()
beta_kds.remove("NTC")
print(f"Batch BETA has {len(beta_kds)} unique TF KDs")

Batch BETA has 7 unique TF KDs


In [ ]:
print(f"Batch BETA TF KDs: {', '.join(beta_kds)}")

Batch BETA TF KDs: SNAPC5, TRAFD1, RBCK1, NAIF1, MTERF4, PIN1, ZBED6


In [ ]:
beta_filt_cells_per_kd = adata_beta_filt.obs.gene_target.value_counts()
beta_filt_cells_per_kd = beta_filt_cells_per_kd.drop("NTC")
beta_filt_cells_per_kd

gene_target
PIN1      78
TRAFD1    77
RBCK1     52
SNAPC5    51
NAIF1     33
MTERF4    32
ZBED6     26
Name: count, dtype: int64

In [ ]:
results_dict_beta = de_analysis_dataset(adata_beta_filt, n_cpus=25, n_reps=2)

Running DE analysis:   0%|          | 0/7 [00:00<?, ?it/s]

/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be poorly estimated by the MAD.
  self.fit_dispersion_prior()
/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:541: UserWarning: As the residual degrees of freedom is less than 3, the distribution of log dispersions is especially asymmetric and likely to be p

In [ ]:
beta_DE_df = build_de_df(results_dict_beta)

In [ ]:
beta_DE_df.head()

,SNAPC5_baseMean,SNAPC5_log2FoldChange,SNAPC5_lfcSE,SNAPC5_stat,SNAPC5_pvalue,SNAPC5_padj,TRAFD1_baseMean,TRAFD1_log2FoldChange,TRAFD1_lfcSE,TRAFD1_stat,...,PIN1_lfcSE,PIN1_stat,PIN1_pvalue,PIN1_padj,ZBED6_baseMean,ZBED6_log2FoldChange,ZBED6_lfcSE,ZBED6_stat,ZBED6_pvalue,ZBED6_padj
A1BG,2.090794,0.657907,1.613191,1.318879,0.187210,0.99514,3.235310,-0.259582,1.013076,-0.531073,...,0.953159,0.174302,0.861628,0.997742,1.999285,-0.548611,1.104251,-1.047380,0.294925,0.998534
A1BG-AS1,NaN,NaN,NaN,NaN,NaN,NaN,0.252513,0.097314,2.543327,0.262381,...,2.238469,0.638538,0.523123,0.997742,NaN,NaN,NaN,NaN,NaN,NaN
A2M,0.690982,0.285339,2.327042,0.769845,0.441392,0.99514,1.239847,0.083991,1.278528,0.219586,...,1.446661,0.345858,0.729449,0.997742,0.506277,-0.338132,1.194558,-0.749747,0.453407,0.998534
A2M-AS1,0.232651,0.094468,2.500232,0.235438,0.813869,0.99514,NaN,NaN,NaN,NaN,...,2.485085,0.284172,0.776278,0.997742,0.234035,-0.102118,1.109997,-0.277981,0.781027,0.998534
A2ML1,1.469387,0.126239,1.343993,0.337321,0.735875,0.99514,2.630036,0.178879,1.150894,0.385241,...,2.238469,0.638538,0.523123,0.997742,0.529991,-0.008621,1.307980,-0.025116,0.979963,0.998534


In [ ]:
count_degs_for_tf_kd_list(beta_DE_df, beta_kds)

SNAPC5 has 0 DEGs
TRAFD1 has 0 DEGs
RBCK1 has 0 DEGs
NAIF1 has 0 DEGs
MTERF4 has 1 DEG
PIN1 has 0 DEGs
ZBED6 has 0 DEGs


In [ ]:
results_dict_beta_3reps = de_analysis_dataset(adata_beta_filt, n_cpus=25, n_reps=3)

Running DE analysis:   0%|          | 0/7 [00:00<?, ?it/s]

/home/ranwyl/pymde_env/lib64/python3.11/site-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)


In [ ]:
beta_DE_df_3reps = build_de_df(results_dict_beta)

In [ ]:
count_degs_for_tf_kd_list(beta_DE_df_3reps, beta_kds)

SNAPC5 has 0 DEGs
TRAFD1 has 0 DEGs
RBCK1 has 0 DEGs
NAIF1 has 0 DEGs
MTERF4 has 1 DEG
PIN1 has 0 DEGs
ZBED6 has 0 DEGs


# Old DE code

Combine all DE results into one DataFrame

In [ ]:
DE_results = pd.concat([alpha_DE_df, beta_DE_df], axis=1)
DE_results = aggregate_DE_df.sort_index()
DE_results.head()

Write out results

In [ ]:
filepath_pkl = "/home/ranwyl/results_tf_project/DE_results_10_2_2025.pkl"
DE_results.to_pickle(filepath_pkl)

Rename genes that are listed by Ensembl ID but have a gene name

In [ ]:
all_genes = DE_results.index.tolist()
ensg_genes = [g for g in all_genes if g.startswith("ENSG")]
print(len(ensg_genes))
print(ensg_genes[:5])

In [ ]:
mg = get_client('gene')
ensembl_results_all = mg.querymany(ensg_genes, fields='symbol', species='human')

Manually search for the genes with duplicate hits on GeneCards. To break ties, use the highest GeneCards Inferred Functionality Score (GIFtS). If there is a tie between highest scoring gene symbols, keep the gene as its Ensembl ID.

In [ ]:
# Change this
dup_hits = {'ENSG00000234352': 'LOC349160', 'ENSG00000249738':'IL12B-AS1', 'ENSG00000257545':'LOC100287944'}

Rename genes

In [ ]:
ensembl_to_gene = dict()
for r in ensembl_results_all:
  if r.get('symbol'):
      ensembl_to_gene[r.get('query')] = r.get('symbol')
ensembl_to_gene.update(dup_hits) # Change duplicate hits
print(len(ensembl_to_gene))

In [ ]:
def rename_ensembl_genes(de_df, ensembl_to_gene_dict):
    """
    Rename genes in DataFrame.
    """
    de_df_renamed = de_df.rename(index=ensembl_to_gene_dict)
    print(f"Converted {len(ensembl_to_gene_dict)} Ensembl IDs to gene symbols")
    return de_df_renamed

In [ ]:
DE_results_renamed = rename_ensembl_genes(DE_results, ensembl_to_gene)

In [ ]:
filepath_pkl = "/home/ranwyl/results_tf_project/DE_results_gene_names_09-2025.pkl"
DE_results_renamed.to_pickle(filepath_pkl)

# Step H: Batch Correction

In [ ]:
adata_alpha = ad.read_h5ad("/home/ranwyl/data_tf_project/Aggregate_ALPHA_Core_Cells.h5ad")
adata_beta = ad.read_h5ad("/home/ranwyl/data_tf_project/Aggregate_BETA_Core_Cells.h5ad")
adata_gamma = ad.read_h5ad("/home/ranwyl/data_tf_project/Aggregate_GAMMA_Core_Cells.h5ad")

In [ ]:
# Filter TF KO and NTC cells
def filter_tf_ko_and_ntc(adata):
    return adata[(adata.obs["gene_target"].isin(tfs)) | (adata.obs["gene_target"] == "NTC")].copy()

# Remove lowly expressed genes based on list
def filter_low_expr_genes(adata, genes_to_keep_list):
    return adata[:,adata.var.index.isin(genes_to_keep_list)].copy()

def filter_cells_and_genes(adata, genes_to_keep_list):
    adata = filter_tf_ko_and_ntc(adata)
    return filter_low_expr_genes(adata, genes_to_keep_list)

# Genes that passed filtering out lowly expressed genes
genes_to_keep = pd.read_pickle("/home/ranwyl/data_tf_project/genes_filtered_30pct_100cells.pkl")
genes_to_keep = genes_to_keep[0].tolist()

In [ ]:
adata_alpha_filtered = filter_cells_and_genes(adata_alpha, genes_to_keep)
adata_beta_filtered = filter_cells_and_genes(adata_beta, genes_to_keep)
adata_gamma_filtered = filter_cells_and_genes(adata_gamma, genes_to_keep)

In [ ]:
adata_combined = ad.concat([adata_alpha_filtered, adata_beta_filtered, adata_gamma_filtered])
adata_combined.X = adata_combined.layers["counts"].copy()
adata_combined

Normalizate to median UMI count of all NTC cells

In [ ]:
median_NTC_UMIs = np.median(qc2._get_ntc_view(adata_combined).obs.n_UMI_counts)
sc.pp.normalize_total(adata_combined, target_sum=median_NTC_UMIs)

Perform log1p transformation and batch correction

In [ ]:
sc.pp.log1p(adata_combined)

# Batch correction via relative z-normalization
a = adata_combined[adata_combined.obs.run == 'ALPHA'].copy()
b = adata_combined[adata_combined.obs.run == 'BETA'].copy()
c = adata_combined[adata_combined.obs.run == 'GAMMA'].copy()
qc2.relative_z_normalization(a)
qc2.relative_z_normalization(b)
qc2.relative_z_normalization(c)

normalized_adata = ad.concat([a,b,c])
normalized_adata

In [ ]:
normalized_adata.write("/home/ranwyl/data_tf_project/Final_Aggregate_TF_KO_NTC_Batch_Normalized.h5ad")